# Fine Tuning Notebook

<img src="https://raw.githubusercontent.com/arad1367/UniLi_sources/main/IMG/logo.jpg"
     alt="University of Liechtenstein"
     width="350"
     height="auto">

### About This Tutorial

This tutorial is prepared by **Dr. Pejman Ebrahimi** for the "Deep Learning and Advanced AI Techniques" course at the University of Liechtenstein.

For more resources and notebooks related to this course, please visit Moodle or GitHub repository: [Course Materials](https://github.com/arad1367/University-of-Liechtenstein/tree/main/Deep%20Learning%20and%20Advanced%20AI).

You can reach out to Dr. Pejman Ebrahimi via email: [pejman.ebrahimi@uni.li](mailto:pejman.ebrahimi@uni.li).

# Evaluating Transformer Models for Text Classification of Cryptocurrency News through a Sustainability Lens

In [1]:
# Login to Hugging face
from huggingface_hub import login
login()

### 1. Install all requirements

In [2]:
# Install transformers
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-y9eq8nfo
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-y9eq8nfo
  Resolved https://github.com/huggingface/transformers to commit 46a4b7c909fab58355936e1c7109bb5e2e558267
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.52.0.dev0-py3-none-any.whl size=11333750 sha256=88fbacca2052a3db7d190d653dabdc2e138a74c577843d3820b1014a9355c7a5
  Stored in directory: /tmp/pip-ephem-wheel-cache-9f67rgvd/wheels/04/a3/f1/b88775f8e1665827525b19ac7590250f1038d947067beba9fb
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3


In [3]:
# Install weights & Bias for model tracking
!pip install wandb

In [2]:
# Install other dependencies
try:
  import datasets, evaluate, accelerate
  import gradio as gr
except ModuleNotFoundError:
  !pip install -U datasets evaluate accelerate gradio # -U --> Update
  import datasets, evaluate, accelerate
  import gradio as gr

import random

import numpy as np
import pandas as pd

import torch
import transformers

print(f"Using transformers version: {transformers.__version__}")
print(f"Using datasets version: {datasets.__version__}")
print(f"Using torch version: {torch.__version__}")

Using transformers version: 4.52.0.dev0
Using datasets version: 3.6.0
Using torch version: 2.6.0+cu124


### 2. Import dataset from Hugging Face
> link to data: https://huggingface.co/datasets/arad1367/sustainability_impact_crypto_data

In [3]:
from datasets import load_dataset
import random

dataset = load_dataset("arad1367/sustainability_impact_crypto_data")

# Check some random data
random_indexs = random.sample(range(len(dataset["train"])), 5)
random_samples = dataset["train"][random_indexs]

print(f"[INFO] Random samples from dataset:\n")
for item in zip(random_samples["text"], random_samples["label"]):
    print(f"Text: {item[0]} | Label: {item[1]}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

crypto_sus_news.csv:   0%|          | 0.00/36.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/300 [00:00<?, ? examples/s]

[INFO] Random samples from dataset:

Text: Cryptocurrency market sees a surge in interest as new investors enter the space amidst market volatility. | Label: neutral
Text: Project CJ partners with national parks to donate a portion of profits toward forest conservation. | Label: positive
Text: Crypto network BM adopts a low-energy protocol that consumes less power than traditional blockchain models. | Label: positive
Text: Critics argue that the cryptocurrency boom is hindering progress on climate goals due to its energy demands. | Label: negative
Text: New analysis shows that crypto mining contributes to significant air pollution in urban areas where facilities are located. | Label: negative


### 3. Text tokenizing function

In [4]:
def tokenize_text(examples):
    """
    Tokenize given example text and return the tokenized text.
    """
    return tokenizer(examples["text"],
                     padding=True,
                     truncation=True)

### Evaluation function

In [5]:
import evaluate
import numpy as np
from typing import Tuple

accuracy_metric = evaluate.load("accuracy")

def compute_accuracy(predictions_and_labels: Tuple[np.array, np.array]):
  """
  Computes the accuracy of a model by comparing the predictions and labels.
  """
  predictions, labels = predictions_and_labels

  # Get highest prediction probability of each prediction if predictions are probabilities
  if len(predictions.shape) >= 2:
    predictions = np.argmax(predictions, axis=1)

  return accuracy_metric.compute(predictions=predictions, references=labels)

In [6]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pejdaniel1000 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

### 5. All steps in one shot


### 5.1. roberta-large-mnli model

In [8]:
# 1. Import necessary packages
import pprint
from pathlib import Path

import numpy as np
import torch

import datasets
import evaluate

from transformers import pipeline
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from scipy.stats import entropy
import time
import psutil
from typing import Dict, List, Union


# wandb.finish()
wandb.init(project="crypto-FT-test", reinit=True)

# 2. Setup variables for model training and saving pipeline
DATASET_NAME = "arad1367/sustainability_impact_crypto_data"
MODEL_NAME = "FacebookAI/roberta-large-mnli"
MODEL_SAVE_DIR_NAME = "models_colab/sustainability_news_roberta-large-mnli_five_epochs"

# 3. Create a directory for saving models
# Note: This will override our existing saved model (if there is one)
print(f"[INFO] Creating directory for saving models: {MODEL_SAVE_DIR_NAME}")
model_save_dir = Path(MODEL_SAVE_DIR_NAME)
model_save_dir.mkdir(parents=True, exist_ok=True)

# 4. Load and preprocess the dataset from Hugging Face Hub
print(f"[INFO] Downloading dataset from Hugging Face Hub, name: {DATASET_NAME}")
dataset = datasets.load_dataset(path=DATASET_NAME)

# Create mappings from id2label and label2id
id2label = {0: "negative", 1: "neutral", 2: "positive"}
label2id = {"negative": 0, "neutral": 1, "positive": 2}

# Create function to map IDs to labels in dataset
def map_labels_to_number(example):
    example["label"] = label2id[example["label"]]
    return example

# Map preprocessing function to dataset
dataset = dataset["train"].map(map_labels_to_number)

# Split the dataset into train/test sets
dataset = dataset.train_test_split(test_size=0.2, seed=42)

# 5. Import a tokenizer and map it to our dataset
print(f"[INFO] Tokenizing text for model training with tokenizer: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=MODEL_NAME,
                                          use_fast=True)

# Create a preprocessing function to tokenize text
def tokenize_text(examples):
    return tokenizer(examples["text"],
                     padding=True,
                     truncation=True)

tokenized_dataset = dataset.map(function=tokenize_text,
                                batched=True,
                                batch_size=1000)

# 6. Set up an evaluation metric & function to evaluate our model
accuracy_metric = evaluate.load("accuracy")

def compute_accuracy(predictions_and_labels):
    predictions, labels = predictions_and_labels

    if len(predictions.shape) >= 2:
        predictions = np.argmax(predictions, axis=1)

    return accuracy_metric.compute(predictions=predictions, references=labels) # note: use "references" parameter rather than "labels"


# 7. Import a model and prepare it for training
print(f"[INFO] Loading model: {MODEL_NAME}")
model = AutoModelForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path=MODEL_NAME,
    num_labels=3,
    id2label=id2label,
    label2id=label2id
)
print(f"[INFO] Model loading complete!")

# Create training arguments
training_args = TrainingArguments(
    output_dir=model_save_dir,
    learning_rate=0.00001,
    weight_decay=0.01,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    use_cpu=False,
    seed=42,
    load_best_model_at_end=True,
    logging_strategy="epoch",
    report_to="wandb",
    hub_private_repo=False,
)

# Model tracking with wandb
wandb.config.update({
    "dataset_name": DATASET_NAME,
    "model_name": MODEL_NAME,
    "learning_rate": training_args.learning_rate,
    "weight_decay": training_args.weight_decay,
    "batch_size": training_args.per_device_train_batch_size,
    "num_epochs": training_args.num_train_epochs,
    "seed": training_args.seed
})

print(f"[INFO] Model tracking started ...")

# Setup Trainer
class MyTrainer(Trainer):
    def log(self, logs, *args, **kwargs):
        # let the parent class handle its own logging signature
        super().log(logs, *args, **kwargs)
        # then send the same metrics to wandb
        import wandb
        wandb.log(logs)

trainer = MyTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_accuracy
)

# 8. Train the model on our text dataset
print(f"[INFO] Commencing model training...")
results = trainer.train()

# 9. Save the trained model (*** note: this will overwrite our previous model ***)
print(f"[INFO] Model training complete, saving model to local path: {model_save_dir}")
trainer.save_model(output_dir=model_save_dir)


# ------------------------ Push Model ------------------------------
# 10. Push the model to the Hugging Face Hub
print(f"[INFO] Uploading model to Hugging Face Hub...")
model_upload_url = trainer.push_to_hub(
    commit_message="Uploading crypto sustainability news text classifier model",
    # token="YOUR_HF_TOKEN_HERE" # requires a "write" HF token
)
print(f"[INFO] Model upload complete, model available at: {model_upload_url}")
# ------------------------ Push Model ------------------------------


# 11. Evaluate the model on the test data
print(f"[INFO] Performing evaluation on test dataset...")
predictions_all = trainer.predict(tokenized_dataset["test"])
prediction_values = predictions_all.predictions
prediction_metrics = predictions_all.metrics

wandb.log(prediction_metrics)

print(f"[INFO] Prediction metrics on the test data:")
pprint.pprint(prediction_metrics)

# Metrics
class ModernEvaluationMetrics:
    def __init__(self, model, tokenizer, dataset, id2label):
        self.model = model
        self.tokenizer = tokenizer
        self.dataset = dataset
        self.id2label = id2label
        self.metrics = {}
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Move model to appropriate device
        self.model = self.model.to(self.device)

        # Initialize modern evaluation metrics
        self.accuracy = evaluate.load("accuracy")
        self.precision = evaluate.load("precision")
        self.recall = evaluate.load("recall")
        self.f1 = evaluate.load("f1")

    def measure_computational_cost(self, batch_size=32):
        """Measures computational cost and efficiency metrics"""
        start_time = time.time()
        start_memory = psutil.Process().memory_info().rss / 1024 / 1024

        texts = self.dataset["test"]["text"][:batch_size]

        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            gpu_memory_before = torch.cuda.memory_allocated() / 1024 / 1024

        inputs = self.tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.model(**inputs)

        end_time = time.time()
        end_memory = psutil.Process().memory_info().rss / 1024 / 1024

        metrics = {
            "inference_time_per_sample": (end_time - start_time) / batch_size,
            "cpu_memory_usage_mb": end_memory - start_memory,
        }

        if torch.cuda.is_available():
            gpu_memory_after = torch.cuda.memory_allocated() / 1024 / 1024
            metrics["gpu_memory_usage_mb"] = gpu_memory_after - gpu_memory_before

        return metrics

    def measure_prediction_confidence(self, logits: np.ndarray) -> Dict[str, Union[float, int]]:
        """Analyzes model confidence and potential hallucination"""
        logits_tensor = torch.tensor(logits).to(self.device)
        softmax_probs = torch.nn.functional.softmax(logits_tensor, dim=1)

        # Move to CPU for numpy operations
        softmax_probs_cpu = softmax_probs.cpu().numpy()
        prediction_entropy = entropy(softmax_probs_cpu, axis=1)
        confidence_scores = softmax_probs.max(dim=1).values.cpu().numpy()

        # Calculate uncertainty threshold using standard deviation
        uncertainty_threshold = np.mean(prediction_entropy) + np.std(prediction_entropy)

        return {
            "mean_confidence": float(np.mean(confidence_scores)),
            "mean_entropy": float(np.mean(prediction_entropy)),
            "high_uncertainty_samples": int(np.sum(prediction_entropy > uncertainty_threshold)),
            "low_confidence_predictions": int(np.sum(confidence_scores < 0.7)),
            "confidence_std": float(np.std(confidence_scores)),
            "entropy_std": float(np.std(prediction_entropy))
        }

    def measure_faithfulness(self, n_samples=100):
        """Measures model faithfulness through input perturbation tests"""
        test_texts = self.dataset["test"]["text"][:n_samples]
        original_predictions = []
        perturbed_predictions = []
        confidence_changes = []

        for text in test_texts:
            # Original prediction
            inputs = self.tokenizer(text, return_tensors="pt", truncation=True)
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

            with torch.no_grad():
                output = self.model(**inputs)
            original_logits = output.logits
            original_pred = torch.argmax(original_logits, dim=1).item()
            original_conf = torch.nn.functional.softmax(original_logits, dim=1).max().cpu().item()
            original_predictions.append(original_pred)

            # Create perturbed text
            words = text.split()
            n_remove = max(1, int(len(words) * 0.2))
            remove_indices = np.random.choice(len(words), n_remove, replace=False)
            perturbed_text = " ".join([w for i, w in enumerate(words) if i not in remove_indices])

            # Perturbed prediction
            inputs = self.tokenizer(perturbed_text, return_tensors="pt", truncation=True)
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

            with torch.no_grad():
                output = self.model(**inputs)
            perturbed_logits = output.logits
            perturbed_pred = torch.argmax(perturbed_logits, dim=1).item()
            perturbed_conf = torch.nn.functional.softmax(perturbed_logits, dim=1).max().cpu().item()

            perturbed_predictions.append(perturbed_pred)
            confidence_changes.append(abs(original_conf - perturbed_conf))

        prediction_stability = np.mean(np.array(original_predictions) == np.array(perturbed_predictions))

        return {
            "prediction_stability": float(prediction_stability),
            "stability_score": float(prediction_stability * 100),
            "mean_confidence_change": float(np.mean(confidence_changes)),
            "confidence_change_std": float(np.std(confidence_changes))
        }

    def measure_answer_correctness(self, predictions: np.ndarray, labels: List[int]) -> Dict:
        """Comprehensive evaluation using modern metrics"""
        pred_labels = np.argmax(predictions, axis=1)

        # Calculate metrics using evaluate library
        accuracy = self.accuracy.compute(predictions=pred_labels, references=labels)
        precision = self.precision.compute(predictions=pred_labels, references=labels, average='weighted')
        recall = self.recall.compute(predictions=pred_labels, references=labels, average='weighted')
        f1 = self.f1.compute(predictions=pred_labels, references=labels, average='weighted')

        # Calculate per-class metrics
        class_metrics = {}
        for idx, class_name in self.id2label.items():
            class_predictions = (pred_labels == idx).astype(int)
            class_labels = (np.array(labels) == idx).astype(int)

            class_metrics[class_name] = {
                "precision": float(self.precision.compute(predictions=class_predictions, references=class_labels)['precision']),
                "recall": float(self.recall.compute(predictions=class_predictions, references=class_labels)['recall']),
                "f1": float(self.f1.compute(predictions=class_predictions, references=class_labels)['f1'])
            }

        return {
            "overall_accuracy": accuracy['accuracy'],
            "weighted_precision": precision['precision'],
            "weighted_recall": recall['recall'],
            "weighted_f1": f1['f1'],
            "per_class_metrics": class_metrics
        }

    def evaluate_all(self):
        """Run all evaluation metrics and return comprehensive results"""
        test_texts = self.dataset["test"]["text"]
        test_labels = self.dataset["test"]["label"]

        inputs = self.tokenizer(test_texts, padding=True, truncation=True, return_tensors="pt")
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.model(**inputs)
        predictions = outputs.logits.cpu().numpy()

        self.metrics["computational_cost"] = self.measure_computational_cost()
        self.metrics["confidence_analysis"] = self.measure_prediction_confidence(predictions)
        self.metrics["faithfulness"] = self.measure_faithfulness()
        self.metrics["correctness"] = self.measure_answer_correctness(predictions, test_labels)

        return self.metrics

# Initialize and run comprehensive evaluation
evaluator = ModernEvaluationMetrics(model, tokenizer, tokenized_dataset, id2label)
evaluation_results = evaluator.evaluate_all()

# Log results to wandb
wandb.log({
    "advanced_metrics": evaluation_results
})

# Print detailed results
print(evaluation_results)

wandb.finish()

[INFO] Creating directory for saving models: models_colab/sustainability_news_roberta-large-mnli_five_epochs
[INFO] Downloading dataset from Hugging Face Hub, name: arad1367/sustainability_impact_crypto_data
[INFO] Tokenizing text for model training with tokenizer: FacebookAI/roberta-large-mnli


Map:   0%|          | 0/60 [00:00<?, ? examples/s]

[INFO] Loading model: FacebookAI/roberta-large-mnli


Some weights of the model checkpoint at FacebookAI/roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
<ipython-input-8-c51ad55cb573>:132: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `MyTrainer.__init__`. Use `processing_class` instead.
  trainer = MyTrainer(


[INFO] Model loading complete!
[INFO] Model tracking started ...
[INFO] Commencing model training...


...

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

[INFO] Model upload complete, model available at: https://huggingface.co/arad1367/sustainability_news_roberta-large-mnli_five_epochs/tree/main/
[INFO] Performing evaluation on test dataset...


[INFO] Prediction metrics on the test data:
{'test_accuracy': 1.0,
 'test_loss': 0.00617571035400033,
 'test_runtime': 0.3477,
 'test_samples_per_second': 172.576,
 'test_steps_per_second': 5.753}


{'computational_cost': {'inference_time_per_sample': 0.0036747828125953674, 'cpu_memory_usage_mb': 0.0, 'gpu_memory_usage_mb': 0.01318359375}, 'confidence_analysis': {'mean_confidence': 0.9938606023788452, 'mean_entropy': 0.03654557093977928, 'high_uncertainty_samples': 5, 'low_confidence_predictions': 0, 'confidence_std': 0.005851067136973143, 'entropy_std': 0.028530379757285118}, 'faithfulness': {'prediction_stability': 1.0, 'stability_score': 100.0, 'mean_confidence_change': 0.012527427077293396, 'confidence_change_std': 0.05751042950043526}, 'correctness': {'overall_accuracy': 1.0, 'weighted_precision': 1.0, 'weighted_recall': 1.0, 'weighted_f1': 1.0, 'per_class_metrics': {'negative': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0}, 'neutral': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0}, 'positive': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0}}}}


epoch,▁▁▃▃▅▅▆▆███
eval/accuracy,▁▇███
eval/loss,█▃▁▁▁
eval/runtime,▁█▂▁▂
eval/samples_per_second,█▁▇█▇
eval/steps_per_second,█▁▇█▇
eval_accuracy,▁▇███
eval_loss,█▃▁▁▁
eval_runtime,▁█▂▁▂
eval_samples_per_second,█▁▇█▇
eval_steps_per_second,█▁▇█▇


### 5.2.distilbert/distilbert-base-uncased

In [ ]:
# 1. Import necessary packages
import pprint
from pathlib import Path

import numpy as np
import torch

import datasets
import evaluate

from transformers import pipeline
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from scipy.stats import entropy
import time
import psutil
from typing import Dict, List, Union


# wandb.finish()
wandb.init(project="crypto-sustainability-news-5-comparison-main", reinit=True)

# 2. Setup variables for model training and saving pipeline
DATASET_NAME = "arad1367/sustainability_impact_crypto_data"
MODEL_NAME = "distilbert/distilbert-base-uncased"
MODEL_SAVE_DIR_NAME = "models_colab/distilbert_sustainability_news_five_epochs"

# 3. Create a directory for saving models
# Note: This will override our existing saved model (if there is one)
print(f"[INFO] Creating directory for saving models: {MODEL_SAVE_DIR_NAME}")
model_save_dir = Path(MODEL_SAVE_DIR_NAME)
model_save_dir.mkdir(parents=True, exist_ok=True)

# 4. Load and preprocess the dataset from Hugging Face Hub
print(f"[INFO] Downloading dataset from Hugging Face Hub, name: {DATASET_NAME}")
dataset = datasets.load_dataset(path=DATASET_NAME)

# Create mappings from id2label and label2id
id2label = {0: "negative", 1: "neutral", 2: "positive"}
label2id = {"negative": 0, "neutral": 1, "positive": 2}

# Create function to map IDs to labels in dataset
def map_labels_to_number(example):
    example["label"] = label2id[example["label"]]
    return example

# Map preprocessing function to dataset
dataset = dataset["train"].map(map_labels_to_number)

# Split the dataset into train/test sets
dataset = dataset.train_test_split(test_size=0.2, seed=42)

# 5. Import a tokenizer and map it to our dataset
print(f"[INFO] Tokenizing text for model training with tokenizer: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=MODEL_NAME,
                                          use_fast=True)

# Create a preprocessing function to tokenize text
def tokenize_text(examples):
    return tokenizer(examples["text"],
                     padding=True,
                     truncation=True)

tokenized_dataset = dataset.map(function=tokenize_text,
                                batched=True,
                                batch_size=1000)

# 6. Set up an evaluation metric & function to evaluate our model
accuracy_metric = evaluate.load("accuracy")

def compute_accuracy(predictions_and_labels):
    predictions, labels = predictions_and_labels

    if len(predictions.shape) >= 2:
        predictions = np.argmax(predictions, axis=1)

    return accuracy_metric.compute(predictions=predictions, references=labels) # note: use "references" parameter rather than "labels"


# 7. Import a model and prepare it for training
print(f"[INFO] Loading model: {MODEL_NAME}")
model = AutoModelForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path=MODEL_NAME,
    num_labels=3,
    id2label=id2label,
    label2id=label2id
)
print(f"[INFO] Model loading complete!")

# Create training arguments
training_args = TrainingArguments(
    output_dir=model_save_dir,
    learning_rate=0.00001,
    weight_decay=0.01,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    use_cpu=False,
    seed=42,
    load_best_model_at_end=True,
    logging_strategy="epoch",
    report_to="wandb",
    hub_private_repo=False,
)

# Model tracking with wandb
wandb.config.update({
    "dataset_name": DATASET_NAME,
    "model_name": MODEL_NAME,
    "learning_rate": training_args.learning_rate,
    "weight_decay": training_args.weight_decay,
    "batch_size": training_args.per_device_train_batch_size,
    "num_epochs": training_args.num_train_epochs,
    "seed": training_args.seed
})

print(f"[INFO] Model tracking started ...")

# Setup Trainer
class MyTrainer(Trainer):
    def log(self, logs, *args, **kwargs):
        # let the parent class handle its own logging signature
        super().log(logs, *args, **kwargs)
        # then send the same metrics to wandb
        import wandb
        wandb.log(logs)

trainer = MyTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_accuracy
)

# 8. Train the model on our text dataset
print(f"[INFO] Commencing model training...")
results = trainer.train()

# 9. Save the trained model (*** note: this will overwrite our previous model ***)
print(f"[INFO] Model training complete, saving model to local path: {model_save_dir}")
trainer.save_model(output_dir=model_save_dir)


# ------------------------ Push Model ------------------------------
# 10. Push the model to the Hugging Face Hub
# print(f"[INFO] Uploading model to Hugging Face Hub...")
# model_upload_url = trainer.push_to_hub(
#     commit_message="Uploading crypto sustainability news text classifier model",
#     # token="YOUR_HF_TOKEN_HERE" # requires a "write" HF token
# )
# print(f"[INFO] Model upload complete, model available at: {model_upload_url}")
# ------------------------ Push Model ------------------------------


# 11. Evaluate the model on the test data
print(f"[INFO] Performing evaluation on test dataset...")
predictions_all = trainer.predict(tokenized_dataset["test"])
prediction_values = predictions_all.predictions
prediction_metrics = predictions_all.metrics

wandb.log(prediction_metrics)

print(f"[INFO] Prediction metrics on the test data:")
pprint.pprint(prediction_metrics)

# Metrics
class ModernEvaluationMetrics:
    def __init__(self, model, tokenizer, dataset, id2label):
        self.model = model
        self.tokenizer = tokenizer
        self.dataset = dataset
        self.id2label = id2label
        self.metrics = {}
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Move model to appropriate device
        self.model = self.model.to(self.device)

        # Initialize modern evaluation metrics
        self.accuracy = evaluate.load("accuracy")
        self.precision = evaluate.load("precision")
        self.recall = evaluate.load("recall")
        self.f1 = evaluate.load("f1")

    def measure_computational_cost(self, batch_size=32):
        """Measures computational cost and efficiency metrics"""
        start_time = time.time()
        start_memory = psutil.Process().memory_info().rss / 1024 / 1024

        texts = self.dataset["test"]["text"][:batch_size]

        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            gpu_memory_before = torch.cuda.memory_allocated() / 1024 / 1024

        inputs = self.tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.model(**inputs)

        end_time = time.time()
        end_memory = psutil.Process().memory_info().rss / 1024 / 1024

        metrics = {
            "inference_time_per_sample": (end_time - start_time) / batch_size,
            "cpu_memory_usage_mb": end_memory - start_memory,
        }

        if torch.cuda.is_available():
            gpu_memory_after = torch.cuda.memory_allocated() / 1024 / 1024
            metrics["gpu_memory_usage_mb"] = gpu_memory_after - gpu_memory_before

        return metrics

    def measure_prediction_confidence(self, logits: np.ndarray) -> Dict[str, Union[float, int]]:
        """Analyzes model confidence and potential hallucination"""
        logits_tensor = torch.tensor(logits).to(self.device)
        softmax_probs = torch.nn.functional.softmax(logits_tensor, dim=1)

        # Move to CPU for numpy operations
        softmax_probs_cpu = softmax_probs.cpu().numpy()
        prediction_entropy = entropy(softmax_probs_cpu, axis=1)
        confidence_scores = softmax_probs.max(dim=1).values.cpu().numpy()

        # Calculate uncertainty threshold using standard deviation
        uncertainty_threshold = np.mean(prediction_entropy) + np.std(prediction_entropy)

        return {
            "mean_confidence": float(np.mean(confidence_scores)),
            "mean_entropy": float(np.mean(prediction_entropy)),
            "high_uncertainty_samples": int(np.sum(prediction_entropy > uncertainty_threshold)),
            "low_confidence_predictions": int(np.sum(confidence_scores < 0.7)),
            "confidence_std": float(np.std(confidence_scores)),
            "entropy_std": float(np.std(prediction_entropy))
        }

    def measure_faithfulness(self, n_samples=100):
        """Measures model faithfulness through input perturbation tests"""
        test_texts = self.dataset["test"]["text"][:n_samples]
        original_predictions = []
        perturbed_predictions = []
        confidence_changes = []

        for text in test_texts:
            # Original prediction
            inputs = self.tokenizer(text, return_tensors="pt", truncation=True)
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

            with torch.no_grad():
                output = self.model(**inputs)
            original_logits = output.logits
            original_pred = torch.argmax(original_logits, dim=1).item()
            original_conf = torch.nn.functional.softmax(original_logits, dim=1).max().cpu().item()
            original_predictions.append(original_pred)

            # Create perturbed text
            words = text.split()
            n_remove = max(1, int(len(words) * 0.2))
            remove_indices = np.random.choice(len(words), n_remove, replace=False)
            perturbed_text = " ".join([w for i, w in enumerate(words) if i not in remove_indices])

            # Perturbed prediction
            inputs = self.tokenizer(perturbed_text, return_tensors="pt", truncation=True)
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

            with torch.no_grad():
                output = self.model(**inputs)
            perturbed_logits = output.logits
            perturbed_pred = torch.argmax(perturbed_logits, dim=1).item()
            perturbed_conf = torch.nn.functional.softmax(perturbed_logits, dim=1).max().cpu().item()

            perturbed_predictions.append(perturbed_pred)
            confidence_changes.append(abs(original_conf - perturbed_conf))

        prediction_stability = np.mean(np.array(original_predictions) == np.array(perturbed_predictions))

        return {
            "prediction_stability": float(prediction_stability),
            "stability_score": float(prediction_stability * 100),
            "mean_confidence_change": float(np.mean(confidence_changes)),
            "confidence_change_std": float(np.std(confidence_changes))
        }

    def measure_answer_correctness(self, predictions: np.ndarray, labels: List[int]) -> Dict:
        """Comprehensive evaluation using modern metrics"""
        pred_labels = np.argmax(predictions, axis=1)

        # Calculate metrics using evaluate library
        accuracy = self.accuracy.compute(predictions=pred_labels, references=labels)
        precision = self.precision.compute(predictions=pred_labels, references=labels, average='weighted')
        recall = self.recall.compute(predictions=pred_labels, references=labels, average='weighted')
        f1 = self.f1.compute(predictions=pred_labels, references=labels, average='weighted')

        # Calculate per-class metrics
        class_metrics = {}
        for idx, class_name in self.id2label.items():
            class_predictions = (pred_labels == idx).astype(int)
            class_labels = (np.array(labels) == idx).astype(int)

            class_metrics[class_name] = {
                "precision": float(self.precision.compute(predictions=class_predictions, references=class_labels)['precision']),
                "recall": float(self.recall.compute(predictions=class_predictions, references=class_labels)['recall']),
                "f1": float(self.f1.compute(predictions=class_predictions, references=class_labels)['f1'])
            }

        return {
            "overall_accuracy": accuracy['accuracy'],
            "weighted_precision": precision['precision'],
            "weighted_recall": recall['recall'],
            "weighted_f1": f1['f1'],
            "per_class_metrics": class_metrics
        }

    def evaluate_all(self):
        """Run all evaluation metrics and return comprehensive results"""
        test_texts = self.dataset["test"]["text"]
        test_labels = self.dataset["test"]["label"]

        inputs = self.tokenizer(test_texts, padding=True, truncation=True, return_tensors="pt")
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.model(**inputs)
        predictions = outputs.logits.cpu().numpy()

        self.metrics["computational_cost"] = self.measure_computational_cost()
        self.metrics["confidence_analysis"] = self.measure_prediction_confidence(predictions)
        self.metrics["faithfulness"] = self.measure_faithfulness()
        self.metrics["correctness"] = self.measure_answer_correctness(predictions, test_labels)

        return self.metrics

# Initialize and run comprehensive evaluation
evaluator = ModernEvaluationMetrics(model, tokenizer, tokenized_dataset, id2label)
evaluation_results = evaluator.evaluate_all()

# Log results to wandb
wandb.log({
    "advanced_metrics": evaluation_results
})

# Print detailed results
print(evaluation_results)

wandb.finish()

[INFO] Creating directory for saving models: models_colab/distilbert_sustainability_news_five_epochs
[INFO] Downloading dataset from Hugging Face Hub, name: arad1367/sustainability_impact_crypto_data
[INFO] Tokenizing text for model training with tokenizer: distilbert/distilbert-base-uncased


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.082300,1.046498,0.716667
2,1.015600,0.978524,0.916667
3,0.944600,0.913298,0.933333
4,0.881800,0.861459,0.933333
5,0.849100,0.841487,0.933333


[INFO] Model training complete, saving model to local path: models_colab/distilbert_sustainability_news_five_epochs
[INFO] Performing evaluation on test dataset...


[INFO] Prediction metrics on the test data:
{'test_accuracy': 0.9333333333333333,
 'test_loss': 0.8414866328239441,
 'test_runtime': 0.0396,
 'test_samples_per_second': 1516.717,
 'test_steps_per_second': 50.557}
{'computational_cost': {'inference_time_per_sample': 0.0006392896175384521, 'cpu_memory_usage_mb': 0.0, 'gpu_memory_usage_mb': 0.01513671875}, 'confidence_analysis': {'mean_confidence': 0.4357568621635437, 'mean_entropy': 1.066909670829773, 'high_uncertainty_samples': 13, 'low_confidence_predictions': 60, 'confidence_std': 0.03926420584321022, 'entropy_std': 0.019179536029696465}, 'faithfulness': {'prediction_stability': 0.95, 'stability_score': 95.0, 'mean_confidence_change': 0.016715476910273235, 'confidence_change_std': 0.01642578028100703}, 'correctness': {'overall_accuracy': 0.9333333333333333, 'weighted_precision': 0.94, 'weighted_recall': 0.9333333333333333, 'weighted_f1': 0.9349567099567099, 'per_class_metrics': {'negative': {'precision': 0.8, 'recall': 0.9230769230769

epoch,▁▁▃▃▅▅▆▆███
eval/accuracy,▁▇███
eval/loss,█▆▃▂▁
eval/runtime,▁▁▂█▁
eval/samples_per_second,██▆▁▇
eval/steps_per_second,██▆▁▇
eval_accuracy,▁▇███
eval_loss,█▆▃▂▁
eval_runtime,▁▁▂█▁
eval_samples_per_second,██▆▁▇
eval_steps_per_second,██▆▁▇
